In [1]:

import logging
import os
import re

from tqdm import tqdm
import pandas as pd
import numpy as np
import spacy
import fitz  # PyMuPDF
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from scipy.stats import entropy

In [2]:
logging.basicConfig(filename='document_processing_errors.log', level=logging.INFO)

In [3]:
def pdf_to_text(path):
    try:
        doc = fitz.open(path)
        text = ""
        for page in doc:
            try:
                text += page.get_text()
            except Exception as page_error:
                print(f"Error extracting text from page in {path}: {page_error}")
                continue
                # Optionally, continue to the next page or log the error
        return text
    except Exception as e:
        logging.info(f"Error processing file {path}: {e}")
        return ""


### Consider using this to only run stuff once through the get_text() procedure:
nOTE THAT THIS  WOULD REPLACE TWO METHODS

In [ ]:
import fitz  # PyMuPDF
import logging

def process_pdf(pdf_path):
    try:
        doc = fitz.open(pdf_path)
    except Exception as e:
        logging.info(f"Error opening PDF {pdf_path}: {e}")
        return None  # Indicate failure to open PDF

    text = ""
    aspect_ratios = []
    for page in doc:
        # Extract text
        try:
            text += page.get_text()
        except Exception as page_error:
            logging.info(f"Error extracting text from page in {pdf_path}: {page_error}")
            # Optionally, continue to the next page or log the error

        # Calculate aspect ratio
        rect = page.rect
        aspect_ratio = rect.width / rect.height
        aspect_ratios.append(aspect_ratio)
    
    # Process aspect ratios for features
    aspect_threshold = detect_significant_change(aspect_ratios)
    features = detect_persistent_changes(aspect_ratios, aspect_threshold)
    num_changes, changes = detect_significant_change_enhanced(aspect_ratios, aspect_threshold)
    stats = extract_aspect_ratio_features(aspect_ratios)
    categories = categorize_aspect_ratios(aspect_ratios)
    
    return {
        "text": text,
        "aspect_ratios": aspect_ratios,
        "page_count": len(doc),
        "aspect_threshold": aspect_threshold,
        "features": features,
        "num_changes": num_changes,
        "changes": changes,
        "stats": stats,
        "categories": categories,
    }


### Use one or the othere here.

In [4]:
nlp = spacy.load("en_core_web_sm")  # Or a larger model as needed

def clean_and_tokenize(text, chunk_size=1000000):
    """
    Tokenizes the text using SpaCy, handling long texts by processing in chunks.
    
    :param text: The text to be tokenized.
    :param chunk_size: Maximum chunk size in characters.
    :return: A string of the lemmatized tokens.
    """
    # Check if the text length exceeds the chunk size
    if len(text) > chunk_size:
        # Initialize an empty list to store tokens from all chunks
        tokens_all_chunks = []
        
        # Process the text in chunks
        for start in range(0, len(text), chunk_size):
            end = start + chunk_size
            # Extract a chunk of text
            chunk = text[start:end]
            # Process the chunk
            doc = nlp(chunk)
            # Extract tokens, lemmatize, and filter as before
            tokens = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]
            tokens_all_chunks.extend(tokens)
        
        # Combine tokens from all chunks and return
        return " ".join(tokens_all_chunks)
    else:
        # If text does not exceed the chunk size, process as before
        doc = nlp(text)
        tokens = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]
        return " ".join(tokens)

# Example of how to apply this function to your DataFrame
# df['tokenized_text'] = df['cleaned_text'].apply(clean_and_tokenize)


In [5]:
def clean_and_tokenize_chunk(chunk):
    """
    Tokenizes a single chunk of text.
    """
    doc = nlp(chunk)
    return [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

def batch_tokenize_texts(texts, batch_size=1000, chunk_size=1000000):
    """
    Tokenize a list of texts in batches, handling long texts by processing in chunks.
    
    :param texts: The list of texts to be tokenized.
    :param batch_size: Number of texts to process in a single batch.
    :param chunk_size: Maximum chunk size in characters for each text.
    :return: A list of lists, where each sublist contains the tokens of a text.
    """
    processed_texts = []
    for text in texts:
        # If the text is longer than chunk_size, split it into chunks
        if len(text) > chunk_size:
            tokens_all_chunks = []
            for start in range(0, len(text), chunk_size):
                end = start + chunk_size
                chunk = text[start:end]
                # Tokenize the chunk and extend the list of tokens
                tokens_all_chunks.extend(clean_and_tokenize_chunk(chunk))
            processed_texts.append(tokens_all_chunks)
        else:
            # For texts that don't exceed the chunk size, process as usual
            tokens = clean_and_tokenize_chunk(text)
            processed_texts.append(tokens)
    
    return processed_texts


# 3. Feature Extraction
- **Combine keyword-matching and TF-IDF**
- **TF-IDF Vectorization:** Use Scikit-learn's TfidfVectorizer to convert the cleaned text documents into a matrix of TF-IDF features.

In [5]:
def check_aspect_ratio_and_mix_feature(pdf_path):
    try:
        doc = fitz.open(pdf_path)
    except Exception as e:
        logging.info(f"Error opening PDF {pdf_path}: {e}")
        raise ValueError("Error opening PDF")
        return [], 0, False  # Returning False as the third value for no significant change
    
    aspect_ratios = []
    
    page_count = len(doc)
    
    for page in doc:
        rect = page.rect
        aspect_ratio = rect.width / rect.height
        aspect_ratios.append(aspect_ratio)
    
    # Detect significant changes in aspect ratio
    aspect_threshold = detect_significant_change(aspect_ratios)
    features = detect_persistent_changes(aspect_ratios, aspect_threshold)
    num_changes,changes = detect_significant_change_enhanced(aspect_ratios, aspect_threshold)
    stats = extract_aspect_ratio_features(aspect_ratios)
    categories = categorize_aspect_ratios(aspect_ratios)
    
    return (aspect_ratios, page_count, aspect_threshold, features, num_changes, changes, stats, categories)

def detect_significant_change(aspect_ratios, threshold=0.1):
    """
    Detects significant changes in aspect ratio.
    :param aspect_ratios: List of aspect ratios for the document's pages.
    :param threshold: The threshold for detecting a significant change.
    :return: True if a significant change is detected, otherwise False.
    """
    for i in range(1, len(aspect_ratios)):
        new_t =  abs(aspect_ratios[i] - aspect_ratios[i-1]) / aspect_ratios[i-1]
        threshold = max(threshold, new_t)

    return threshold


def detect_significant_change_enhanced(aspect_ratios, threshold=0.1):
    changes = np.diff(aspect_ratios) / aspect_ratios[:-1]
    mean_change = np.mean(changes)
    std_dev_change = np.std(changes)
    
    significant_changes = changes[(changes > mean_change + std_dev_change * threshold) | 
                                  (changes < mean_change - std_dev_change * threshold)]
    return len(significant_changes) > 0, significant_changes



def detect_persistent_changes(aspect_ratios, change_threshold=0.1, persistence_threshold=3):
    """
    Detects persistent changes in aspect ratios.
    
    :param aspect_ratios: List of aspect ratios for each page in the document.
    :param change_threshold: The minimum change in aspect ratio to consider.
    :param persistence_threshold: The minimum number of consecutive pages over which a change must persist to be considered.
    :return: A feature set capturing aspects of persistent changes.
    """
    changes = [abs(aspect_ratios[i] - aspect_ratios[i-1]) / aspect_ratios[i-1] for i in range(1, len(aspect_ratios))]
    persistent_changes = 0
    current_persistence = 0
    
    for change in changes:
        if change > change_threshold:
            current_persistence += 1
        else:
            if current_persistence >= persistence_threshold:
                persistent_changes += 1
            current_persistence = 0
    
    # Catch any sequence that goes until the end of the document
    if current_persistence >= persistence_threshold:
        persistent_changes += 1
    
    # Example features: count of persistent changes, presence of any persistent change
    features = {
        'persistent_change_count': persistent_changes,
        'has_persistent_change': int(persistent_changes > 0)
    }
    
    return features



def extract_aspect_ratio_features(aspect_ratios):
    """
    Extracts statistical features from a list of aspect ratios.
    
    :param aspect_ratios: List of aspect ratios for the document's pages.
    :return: Dictionary of statistical features.
    """
    if not aspect_ratios:  # Check if the list is empty
        return {
            'mean': 0,
            'std': 0,
            'min': 0,
            'max': 0
        }
    
    aspect_ratios_array = np.array(aspect_ratios)
    return {
        'mean': np.mean(aspect_ratios_array),
        'std': np.std(aspect_ratios_array),
        'min': np.min(aspect_ratios_array),
        'max': np.max(aspect_ratios_array)
    }



def categorize_aspect_ratios(aspect_ratios):
    """
    Categorizes aspect ratios as portrait, landscape, or square.
    
    :param aspect_ratios: List of aspect ratios for the document's pages.
    :return: List of categories corresponding to each aspect ratio.
    """
    categories = []
    for ar in aspect_ratios:
        if ar < 0.95:
            categories.append('portrait')
        elif ar > 1.05:
            categories.append('landscape')
        else:
            categories.append('square')
    return categories



In [6]:

def cluster_aspect_ratios(aspect_ratios, n_clusters=3):
    """
    Clusters aspect ratios into n clusters.
    
    :param aspect_ratios: List of aspect ratios for the document's pages.
    :param n_clusters: Number of clusters to form.
    :return: Labels for each page indicating cluster membership.
    """
    aspect_ratios = np.array(aspect_ratios).reshape(-1, 1)  # Reshape for clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(aspect_ratios)
    return kmeans.labels_



def calculate_features(cluster_assignments):
    features = []
    for document in cluster_assignments:
        # Feature 1: Cluster Transition Indicator (CTI)
        transitions = sum([1 for i in range(1, len(document)) if document[i] != document[i-1]])
        
        # Feature 2: Cluster Distribution Spread (CDS)
        cluster_counts = np.bincount(document, minlength=3)  # Assuming 3 clusters
        proportions = cluster_counts / np.sum(cluster_counts)
        spread = entropy(proportions)  # Using entropy as a measure of spread
        
        features.append((transitions, spread))
    
    return features


In [7]:
def detect_outliers_z_score(aspect_ratios, threshold=2):
    aspect_ratios = np.array(aspect_ratios).flatten()  # Ensures aspect_ratios is 1D
    mean_ar = np.mean(aspect_ratios)
    std_ar = np.std(aspect_ratios)
    outliers = [i for i, ar in enumerate(aspect_ratios) if abs((ar - mean_ar) / std_ar) > threshold]
    
    return outliers


In [8]:
np.seterr(divide='ignore', invalid='ignore')

def calculate_text_density(pdf_path):
    """
    Calculates text density (words per page) for a PDF document.
    
    :param pdf_path: Path to the PDF document.
    :return: List of text densities for each page.
    """
    doc = fitz.open(pdf_path)
    text_densities = []
    for page in doc:
        text = page.get_text("text")
        word_count = len(text.split())
        area = page.rect.width * page.rect.height
        text_density = word_count / area if area else 0
        text_densities.append(text_density)
    return text_densities

def correlate_aspect_ratio_text_density(aspect_ratios, text_densities):
    """
    Correlates aspect ratios with text densities.
    
    :param aspect_ratios: List of aspect ratios for the document's pages.
    :param text_densities: List of text densities for each page.
    :return: Correlation coefficient between aspect ratios and text densities.
    """
    if len(aspect_ratios) != len(text_densities):
        raise ValueError("Aspect ratios and text densities lists must be of the same length.")
    
    # Check standard deviation to decide on correlation calculation
    std_aspect_ratios = np.nanstd(aspect_ratios)
    std_text_densities = np.nanstd(text_densities)
    
    if std_aspect_ratios > 0 and std_text_densities > 0:
        correlation = np.corrcoef(aspect_ratios, text_densities)[0, 1]
    else:
        correlation = np.nan 
    
    return correlation


In [9]:
# Function to check for keyword presence
def check_keywords(text, keyword_list):
    text = text.lower()
    return int(any(keyword in text for keyword in keyword_list))


# alternate code below

In [ ]:


def check_keywords_alternate(text, keyword_list):
    text = text.lower()
    # Create a pattern that matches whole words only, for all keywords
    pattern = r'\b(' + '|'.join([re.escape(keyword) for keyword in keyword_list]) + r')\b'
    return int(bool(re.search(pattern, text)))

def safety_not_run_thing():
    # Convert keywords to lowercase for case-insensitive matching
    keywords = {category: [keyword.lower() for keyword in keyword_list] for category, keyword_list in keywords.items()}

    # Assuming 'tokenized_text' contains space-separated tokens, it should work well with the modified check_keywords function.
    # Just ensure 'tokenized_text' is a string; if it's a list of tokens, you might need to join them first:
    # df['tokenized_text_str'] = df['tokenized_text'].apply(' '.join)

    for category, keyword_list in keywords.items():
        df[category + '_keyword'] = df['tokenized_text'].apply(check_keywords, args=(keyword_list,))


In [10]:
def combine_tfidf_keyword(df):
    # Step 2: TF-IDF Calculation
    vectorizer = TfidfVectorizer(max_features=5000)  # Adjust number of features as needed
    tfidf_matrix = vectorizer.fit_transform(df['tokenized_text'])

    # Step 3. Combine keyword and tfidf features into a single matrix    
    # Convert binary keyword matches to a matrix
    keyword_features = df[[col for col in df.columns if '_keyword' in col]].to_numpy()
    # Combine TF-IDF features with keyword binary indicators
    combined_features = np.hstack((tfidf_matrix.toarray(), keyword_features))

    # Now `combined_features` is ready for model training, and should be aligned with your labels.
    return combined_features


# 4. Training the Classification Model

### Next Steps (not-implemented)

    Train Your Model: Use the combined_features matrix along with your labels to train and evaluate your classification model.
    Evaluation and Refinement: Assess the model's performance and adjust your keyword lists, TF-IDF parameters, or model choice as needed.

This approach leverages both the specificity of keyword matching and the nuanced importance scoring of TF-IDF, providing a rich set of features for document classification.

- **Splitting Data:** Use your 1000 classified documents as training data. Ensure you have a balanced dataset for the three categories.

In [ ]:
def split_data(X, y):
    # Split the data - 70% for training, 30% for testing; adjust ratios as you see fit
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    return X_train, X_test, y_train, y_test



- **Model Selection and Training:** Given the textual nature of your task, models like CNN or LSTM could perform well. TensorFlow/Keras will be used here.

# An initial simple Binary logistic  regression

In [ ]:
def train_logistic_regression(X_train, y_train):

    # Initialize the Logistic Regression model
    model = LogisticRegression(max_iter=1000)  # Increasing max_iter for convergence

    # Train the model on the training data
    model.fit(X_train, y_train)

    # Return the trained model
    return model



# Not using the NN below yet


In [ ]:
def train_nn(X_train, y_train, X_test, y_test):
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Embedding, LSTM

    model = Sequential()
    model.add(Embedding(input_dim=5000, output_dim=64))  # Adjust according to the TF-IDF feature size
    model.add(LSTM(128))
    model.add(Dense(3, activation='softmax'))  # Three categories

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5)
    return model

# 5. Classification of New Documents --- not ready:
- **Predicting Categories:** Use the trained model to predict categories for new documents after preprocessing and vectorization.

In [ ]:
def predict_category(text):
    clean_text = clean_and_tokenize(text)
    vectorized_text = vectorizer.transform([clean_text])
    prediction = model.predict(vectorized_text)
    return prediction

# 6. Evaluation and Iteration not ready

- **Evaluation:** Use metrics like accuracy, precision, recall, and F1 score to evaluate your model on the test set.

In [ ]:
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

- **Iteration:** Based on evaluation results, iterate over your model by tuning hyperparameters, trying different models (e.g., BERT for text classification), or using more advanced text vectorization techniques.

# 7. Scalability and Optimization

- Consider parallel processing or distributed computing for preprocessing steps if you face performance bottlenecks.
- Explore incremental learning or online learning models if retraining on new data frequently.


In [64]:
df.columns

Index(['link', 'presentation', 'fname', 'cleaned_text', 'tokenized_text'], dtype='object')

# Implmentation

In [4]:
import os
import pandas as pd
dataset_path = '../../data/dataset.csv'
# DATA_PATH = '/dave/presentations/'
DATA_PATH = '/home/mike/Downloads'

dfpickle_path = "/dave/data/df.pkl"
force = False
if not os.path.exists(dfpickle_path) or force:
    print("Warning, is dave mounted?")
else:
    print('Going to load df from pickle file')


Going to load df from pickle file


# Careful  with this stored object
* it takes nearly 2 hours to generate it
* Not that is is the df that include cleaned_text and tokenized_text-- but none of the features generated below

In [5]:
if os.path.exists(dfpickle_path) and not force:
    df = pd.read_pickle(dfpickle_path)
else:
    df = pd.read_csv(dataset_path, header=0)
    df['fname'] = DATA_PATH + df['fname']

    tqdm.pandas(desc="Processing documents")
    df['cleaned_text'] = df['fname'].progress_apply(pdf_to_text)

    df['tokenized_text'] = df['cleaned_text'].progress_apply(clean_and_tokenize)

## Next are the features anticipated to identify mix docs around aspect_ratio, text_density and stuff

In [6]:
df.head()

,link,presentation,fname,cleaned_text,tokenized_text
0,https://www.sec.gov/Archives/edgar/data/134547...,1,/dave/presentations/p23-0016_exhibit1.pdf,Confidential – Not for Reproduction or Distrib...,confidential Reproduction Distribution present...
1,https://www.sec.gov/Archives/edgar/data/129970...,1,/dave/presentations/jan2024deck.pdf,"Axos Financial, Inc. \nInvestor Presentation\n...",Axos Financial Investor Presentation January N...
2,https://www.sec.gov/files/hamilton-lane-presen...,1,/dave/presentations/hamilton-lane-presentation...,SEC Asset Management Advisory Committee\nSepte...,SEC Asset Management Advisory Committee Septem...
3,https://www.sec.gov/files/amac-emoney-presenta...,1,/dave/presentations/amac-emoney-presentation-0...,1\nThe Power of \nFinancial Planning \nTechnol...,Power Financial Planning Technology Presentati...
4,https://www.sec.gov/files/amac-kitces-presenta...,1,/dave/presentations/amac-kitces-presentation-0...,"1\nMichael E. Kitces\nMSFS, MTAX, CFP®, CLU, C...",Michael Kitces MSFS MTAX CFP CLU ChFC RHU REBC...


In [23]:
def append_data_or_nan(a_list, data):
    try:
        a_list.append(data)
    except Exception as e:
        logging.info(e)
        a_list.append(np.nan)

In [25]:
# Assuming `df` is your DataFrame and it has a column `pdf_path` pointing to each PDF file.

# Initialize empty lists to store your new features
aspect_ratio_means = []
aspect_ratio_std = []
aspect_ratio_min = []
aspect_ratio_max = []
page_counts = []
aspect_thresholds = []
persistent_changes = []
persistent_change_counts = []
num_ar_changes = []
significant_ar_changes = []
text_density_means = []
text_density_correlations = []
categories_lists = []
portrait_counts = []
landscape_counts = []
square_counts = []


outliers_counts = []
cluster_labels = []
cluster_transitions = []
cluster_spreads = []

for pdf_path in tqdm(df['fname'], desc="Processing PDFs"):
    try:
        logging.info(f"Processing aspect stuff for {pdf_path}")
        # Run your feature extraction functions
        (
            aspect_ratios,
            page_count,
            aspect_threshold,
            features,
            num_changes,
            changes,
            stats,
            categories,
        ) = check_aspect_ratio_and_mix_feature(pdf_path)
        
        text_densities = calculate_text_density(pdf_path)
        correlation = correlate_aspect_ratio_text_density(aspect_ratios, text_densities)
        outliers = detect_outliers_z_score(aspect_ratios)
        cluster_label = cluster_aspect_ratios(aspect_ratios)
    except Exception as e:
        logging.error(f"Error processing {pdf_path}: {e}")
        
        # For simplicity, let's just use some of the features as examples
    append_data_or_nan(aspect_ratio_means, stats['mean'])
    append_data_or_nan(aspect_ratio_std, stats['std'])
    append_data_or_nan(aspect_ratio_min, stats['min'])
    append_data_or_nan(aspect_ratio_max, stats['max'])
    append_data_or_nan(page_counts, page_count)
    append_data_or_nan(aspect_thresholds, aspect_threshold)
    append_data_or_nan(persistent_changes, features['has_persistent_change'])
    append_data_or_nan(persistent_change_counts, features['persistent_change_count'])
    append_data_or_nan(num_ar_changes, num_changes)
    append_data_or_nan(significant_ar_changes, changes)
    append_data_or_nan(text_density_means, np.mean(text_densities))
    append_data_or_nan(text_density_correlations, correlation)
    append_data_or_nan(categories_lists, categories)  # This one is a bit tricky as it's a list. Might aggregate or process further.
    

    
    
    
    append_data_or_nan(outliers_counts, len(outliers) )  # Assuming aspect ratios are recalculated within the function
    append_data_or_nan(cluster_labels,cluster_label)
    
portrait_counts = [cats.count('portrait') if isinstance(cats, list) else 0 for cats in categories_lists]
landscape_counts = [cats.count('landscape') if isinstance(cats, list) else 0 for cats in categories_lists]
square_counts = [cats.count('square') if isinstance(cats, list) else 0 for cats in categories_lists]
cluster_features = calculate_features(cluster_labels)
cluster_transitions =[x[0] for x in cluster_features]
cluster_spreads = [x[1] for x in cluster_features]
    



Processing PDFs:   0%|          | 0/2000 [22:12<?, ?it/s]


KeyboardInterrupt: 

In [18]:


# Now add these lists as columns to your DataFrame
df['aspect_ratio_means'] = aspect_ratio_means
df['aspect_ratio_std'] =  aspect_ratio_std
df['aspect_ratio_min'] = aspect_ratio_min
df['aspect_ratio_max'] = aspect_ratio_max
df['page_counts'] = page_counts
df['aspect_thresholds'] = aspect_thresholds
df['persistent_changes'] = persistent_changes
df['persistent_change_counts'] = persistent_change_counts
df['num_ar_changes'] = num_ar_changes
df['significant_ar_changes'] = significant_ar_changes
df['text_density_means'] = text_density_means
df['text_density_correlations'] = text_density_correlations
df['portrait_count'] = portrait_counts
df['landscape_count'] = landscape_counts
df['square_count'] = square_counts
# For categories, you might need to process the lists into something usable as a feature. Example below.
df['outliers_counts'] = outliers_counts
df['cluster_transitions'] = cluster_transitions
df['cluster_spreads'] = cluster_spreads



ValueError: Length of values (0) does not match length of index (2000)

In [15]:
outliers_counts

[]

In [23]:
kmeans_labels = cluster_aspect_ratios(aspect_ratios)
print(f"kmeans_labels:  {kmeans_labels}")

kmeans_labels:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


/uw/.venvs/matchnames/lib/python3.10/site-packages/sklearn/base.py:1474: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


In [15]:
outliers = detect_outliers_z_score(aspect_ratios)
print(f"outliers: {outliers}")

outliers: []


In [25]:
text_densities = calculate_text_density(testit)
correlation = correlate_aspect_ratio_text_density(aspect_ratios, text_densities)
print(f"text_densities: {text_densities}")
print(f"correlation: {correlation}")

text_densities: [9.696639598600383e-05, 0.0016071664355978082, 0.00184029840892586, 0.0006148082128474285, 0.0001258500033009837, 0.0006024295239981514, 0.0006189344424638542, 0.0011017033075856606, 0.000662259853436324, 0.000662259853436324, 0.0008252459232851389, 0.00089126559714795, 0.0003816762395193768, 0.0006395655905459827, 0.000445632798573975, 0.0005652934574503202, 0.0005508516537928303, 0.0004002442727932924, 0.0008231828084769261, 0.00037136066547831253, 0.000445632798573975, 0.0006416287053541955, 0.0007241532976827095, 0.0002867729583415858, 0.00031153033604013996, 0.0004930844391628705, 0.0006333762461213442, 0.0005425991945599789, 0.0006313131313131314, 0.0003362877137386941, 0.00024757377698554167, 0.0003300983693140556, 0.0004023073876015052, 0.0003878655839440153, 0.0005570409982174688]
correlation: -2.8941088484809674e-16


In [ ]:
keywords = {
    "financial_terms": ['financial', 'investment', 'share price', 'financial metrics', 'investment strategy'],
    "legal_statements": ['confidentiality statement', 'legal disclaimer', 'disclosure statement', 'proprietary information', 'intellectual property'],
    "company_info": ['company overview', 'company analysis', 'business model', 'company performance'],
    "presentation_content": ['visual aids', 'data charts', 'case studies', 'comparative analysis'],
    "company_targets": ['sales targets', 'company targets', 'performance targets'],
    "financial_discussions": ['financial figures', 'financial projections', 'financial results', 'financial language'],
    "regulatory_references": ['SEC filings', 'regulatory filings', 'external entities', 'lawsuits'],
    "detail_descriptions": ['loan details', 'product details', 'research and development', 'financial details'],
    "company_specific": ['company specific', 'industry specific', 'company-specific analysis', 'specific company focus']
    # "Other Clusters" category is omitted since it's broad and without specific keywords
}


# Apply keyword matching
for category, keyword_list in keywords.items():
    df[category + '_keyword'] = df['tokenized_text'].apply(check_keywords, args=(keyword_list,))


In [ ]:
features = combine_tfidf_keyword(df)


In [ ]:
X_train, X_test, y_train, y_test = split_data(features, df['presentation'])

In [ ]:
model = train_logistic_regression(X_train, y_train)

In [ ]:
# Predict the labels for the test set
y_pred = model.predict(X_test)

# Calculate and print the evaluation metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average='binary'))
print("Recall:", recall_score(y_test, y_pred, average='binary'))
print("F1 Score:", f1_score(y_test, y_pred, average='binary'))


In [ ]:
y_test.shape

In [ ]:
import pandas as pd

# Assuming you have a DataFrame `df_test` corresponding to your test dataset
# And it includes a column 'doc_id' or similar that uniquely identifies each document
# If you don't have such a DataFrame, you can create it from `X_test` and `y_test`


type(X_test), type(y_test)
# df_test = pd.DataFrame({'doc_id': X_test.index, 'text': X_test, 'label': y_test})
# df_test

In [ ]:

# First, ensure `X_test` retains its index after splitting so you can merge based on index
misclassified_df = pd.DataFrame({
    'True Label': y_test,
    'Predicted Label': y_pred
})

# If `X_test` and `y_test` don't automatically align, you might need to reset the index
# misclassified_df = misclassified_df.reset_index()

# Add a column to indicate whether each prediction is correct
misclassified_df['Correctly Classified'] = misclassified_df['True Label'] == misclassified_df['Predicted Label']

# Filter the DataFrame to only include misclassified documents
misclassified_docs = misclassified_df[~misclassified_df['Correctly Classified']]

# Optionally, join with the original DataFrame (df) to include text or other identifying information
# This step requires that `df` and `misclassified_docs` can be aligned by index or a unique identifier
# Example:
# misclassified_docs = misclassified_docs.join(df[['doc_id', 'text']], how='left')

misclassified_docs
